# 第10回 その2: ソフトマックス関数を用いた回帰での2クラス分類
10_01_softmax_3class.ipynbではロジスティック回帰をソフトマックス関数を用いて改造することで他クラス分類可能にしました。この方式を使って2クラス分類した場合にどうなるか動作を確認してみます。  

## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/10"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/10/
# 現在のフォルダの中身を表示
%ls

`2class_data.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込みと標準化
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`2class_data.csv` を読み込みます。  
このデータは第8回でも使用した2クラスデータです。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('2class_data.csv', encoding='SHIFT-JIS')

# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# x1とx2のデータを抽出し，numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:, ['x1','x2']].to_numpy()

# クラス番号を抽出し，numpy用データ(ndarray型) に変換する。
Y = csv_data.loc[:,'class'].to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

# クラス数を得る。
num_classes = int(np.max(Y) + 1)
print('Number of classes: ' + str(num_classes))

標準化を行い，データをプロットします。

In [ ]:
# データを標準化する。
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 以下は color_list = ['b', 'r'] とほぼ同じ。
# 光の三原色(赤,緑,青)の値それぞれを0～1の範囲で定義している。
color_list = [(0, 0, 1), (1, 0, 0)]

# データをプロット
plt.figure(figsize=(7,7))
for k in range(num_classes):
  # n番目のクラスに属している(Y==c)データをプロット
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlim([-2.5, 2.5])
plt.ylim([-2.5, 2.5])
plt.show()

## ステップ2: ソフトマックス関数を用いた多クラス識別の実行  
`09_01_softmax_3class.ipynb`で定義した，ソフトマックス関数や多クラス識別関数，勾配計算の関数を以下に定義しておきます。  

In [ ]:
def softmax(z):
  '''
      ソフトマックス関数
      z: 要素数=クラス数のベクトル
      y_hat: クラス毎の確率。要素数=クラス数のベクトル
  '''
  return (np.exp(z) / np.sum(np.exp(z)))

def multiclass_regression(x, W, b):
  '''
      1サンプルデータに対してクラス分類を行う
      x: 1サンプルデータ。要素数=次元数のベクトル
      W: クラス毎に定義されている，各次元に対する重み。サイズ=[クラス数 x 次元数]の行列。
         行列Wのk行目のベクトルwkにはk番目のクラスの確率を計算するための重みが格納される。
      b: クラス毎に定義されている，切片（バイアス）成分。サイズ=クラス数のベクトル。
      y_hat: ソフトマックス回帰の出力。要素数=クラス数のベクトル
  '''

  # 行列Wとベクトルxの内積はベクトル
  # z1 = w11*x1 + w12*x2 + ... + b1
  # z2 = w21*x1 + w22*x2 + ... + b2
  # ...
  z = np.dot(W, x) + b

  # ソフトマックス関数に通す
  # zおよびy_hatはベクトル
  y_hat = softmax(z)

  return y_hat


In [ ]:
def cross_entropy(y_hat, y):
  '''
      クロスエントロピー損失を計算する。
      y_hat: マルチクラス回帰によって推定された y
      y: 正解のクラス
  '''
  # 正解クラスの確率を1，それ以外のクラスの確率を0とするベクトル（one-hotベクトル）を作成する。
  y_onehot = np.zeros(num_classes)
  y_onehot[y] = 1.0

  ce = -1.0 * np.sum(y_onehot*np.log(y_hat))

  return ce

def calc_gradient(y_hat, y, x):
  '''
      勾配を計算する
      y_hat: マルチクラス回帰によって推定された y。サイズ=Kのベクトル
      y: 正解のクラス。スカラ
      x: データ。サイズ=次元数のベクトル
      grad_W: Wの勾配：wの数だけ存在。つまりサイズ=(KxD)の行列
      grad_b: bの勾配：bの数だけ存在。つまりサイズ=Kのベクトル
      ただし，D:次元数，K:クラス数である。
  '''
  # 次元数D, クラス数Kを得る。
  D = np.size(x)
  K = np.size(y_hat)

  # 正解クラスの確率を1，それ以外のクラスの確率を0とするベクトル（one-hotベクトル）を作成する。
  y_onehot = np.zeros(K)
  y_onehot[y] = 1.0

  # 勾配を計算
  grad_W = np.zeros([K, D])
  grad_b = np.zeros(K)
  for k in range(K):
    grad_W[k,:] = (y_hat[k] - y_onehot[k]) * x
    grad_b[k] = y_hat[k] - y_onehot[k]

  return grad_W, grad_b

重み行列 $\bf W$ と切片（バイアス）成分 $\bf b$ の初期値を決めます。  
ロジスティック回帰のときと同様，各線形式の切片成分 $\bf b$ の初期値はゼロとし，重み行列 $\bf W$ の初期値は正規分布に従う乱数で決めます。

In [ ]:
np.random.seed(0)

# 正規分布に従う乱数行列
initial_W = np.random.randn(num_classes, num_dimensions)
# ゼロベクトル
initial_b = np.zeros(num_classes)

print('initial_W = ')
print(initial_W)
print('initial_b = ')
print(initial_b)

では`09_01_softmax_3class.ipynb`と同様に，全てのデータを用いて繰り返し更新しましょう。   
ここでは全データサンプルに対して更新を行うひとまとまりの処理を20周（エポック）行っています。

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

np.random.seed(0)

# 学習率
lr = 0.2

# 重みおよびバイアスの初期化
W = initial_W.copy()
b = initial_b.copy()

# 描画
color_list = [(0, 0, 1), (1, 0, 0)]
fig = plt.figure(figsize=(7,7))
# データの描画
for k in range(num_classes):
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
images = []

# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

for epoch in range(30):
  
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n,:]
    y = Y_tmp[n]
  
    # 回帰関数に与える
    y_hat = multiclass_regression(x, W, b)
  
    # 勾配の計算
    grad_W, grad_b = calc_gradient(y_hat, y, x)

    # 更新スピードの速い1エポックでは50サンプルごと，それ以外は200サンプル
    # ごとにアニメーション描画する。
    if (epoch < 2 and n % 50 == 0) or (epoch >=2 and n % 200 == 0):
      img = []
      x1 = np.linspace(-3, 3)
      for k in range(num_classes):
        x2 = -1.0 * (b[k] + W[k, 0]*x1) / W[k, 1]
        img += plt.plot(x1, x2, c=color_list[k])
      img.append(plt.text(-2.3, 2.3, 'epoch: '+str(epoch), size='x-large'))
      images.append(img)

    # 更新
    W -= lr * grad_W
    b -= lr * grad_b

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y)
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))

plt.legend()
plt.xlim([-2.5, 2.5])
plt.ylim([-2.5, 2.5])

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=200)
# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)

エポック毎の損失関数（クロスエントロピー）および分類正解率をプロットします。  

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

（エポック毎に計算していますが）学習完了後の分類正解率を計算します。  

In [ ]:
# 正解率
accuracy = 0

for n in range(num_samples):
  x = X[n,:]
  y = Y[n]
  # マルチクラス回帰関数に与える
  y_hat = multiclass_regression(x, W, b)

  if np.argmax(y_hat) == y:
    accuracy += 1

accuracy = 100.0 * accuracy / num_samples

print('Accuracy = %.3f' % (accuracy))

100%の正解率が得られました。

最後に，クラス毎の識別境界を可視化します。  


In [ ]:
# x1，x2ごとの格子点の数
grid_size = 200

# 格子点：-2.6から2.6まで200個分，等間隔に区切った数列
grid = np.linspace(-2.6, 2.6, grid_size)

# クラス識別結果を色塗りする領域：200x200の図
# 最後の3は，色情報(RGB=赤・緑・青の3次元ごとの色の強さ)
image = np.zeros([grid_size, grid_size, 3])

color_list = [(0, 0, 1), (1, 0, 0)]

for i in range(grid_size):
  for j in range(grid_size):
    # 格子点のデータ
    x = np.array([grid[j], grid[i]])
    # 格子点データを識別する
    y_hat = multiclass_regression(x, W, b)
    label = np.argmax(y_hat)
    # 識別結果のクラスの色で，色を塗る
    # (グラフは原点が左下だが，画像は左上が原点なので，上下反転させるために-iとしている)
    image[-i, j, :] = color_list[label]

plt.figure(figsize=(7,7))
# 色塗り結果を表示
plt.imshow(image, alpha=0.4, extent=(-2.6, 2.6, -2.6, 2.6))
# データおよび補助線を表示
for k in range(num_classes):
  x2 = -1.0 * (b[k] + W[k, 0]*x1) / W[k, 1]
  plt.plot(x1, x2, c=color_list[k])
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-2.5, 2.5])
plt.ylim([-2.5, 2.5])
plt.show()

赤の補助線と青の補助線の間に赤クラスと青クラスの境界線が作成されていることが確認できます。  

ソフトマックス関数を使った場合でもロジスティック回帰で行ったような2クラス分類が行えることが分かりました。  
クラス数に依らず使用できるという点では，ロジスティック回帰よりもこの方法の方が万能ということになります。  
（ただしクラス数が2に限定した問題を解くのであれば，ロジスティック回帰の方がパラメータ数が少なくてシンプルという利点はあります。）